# Combine the data computed for Jeffreson+LtU, 2024 using the PRFMData class in the parent directory

In [6]:
import numpy as np
import pickle
from pathlib import Path
import glob, sys
sys.path.append('../')

import configparser
config = configparser.ConfigParser()
config.read('./config/config_pressures-SFRs.ini')

['./config/config_pressures-SFRs.ini']

In [7]:
tags = ['ETG-vlM', 'ETG-lowM', 'ETG-medM', 'ETG-hiM', 'MW', 'NGC300']
Rmaxs_kpc = [config[tag].getfloat('RMAX') for tag in tags]
Rmaxs_kpc = [1.5 if elem==2. else elem for elem in Rmaxs_kpc] # cut to actual max size of ETGs

# simulation data

In [10]:
dicts = [{}, {}, {}, {}, {}, {}]
for tag, dict in zip(tags, dicts):
    filesavedir = Path(config[tag]['ROOT_DIR']) / config[tag]['SUBDIR']

    # weight from this array (temperature threshold here has no impact)
    filesavename = min(glob.glob(str(filesavedir / "weights_*.pkl")))
    with open(filesavename, "rb") as f:
        props_3D = pickle.load(f)
    for key in props_3D.keys():
        dict[key] = props_3D[key]

    # weight from this array (temperature threshold here has no impact)
    filesavename = min(glob.glob(str(filesavedir / "Figs-10-11_*_T2.0e+04_avir2.0e+00.pkl")))
    with open(filesavename, "rb") as f:
        props_3D = pickle.load(f)
    for key in props_3D.keys():
        dict[key] = props_3D[key]

    # pressures, counts and SFRs from this array (respond to temperature threshold)
    filesavename = min(glob.glob(str(filesavedir / "pressures-SFRs-fixed-mp_*_T2.0e+04_avir2.0e+00.pkl")))
    with open(filesavename, "rb") as f:
        props_3D = pickle.load(f)
    for key in props_3D.keys():
        dict[key] = props_3D[key]
    
    # radial bin data from this array
    filesavename = min(glob.glob(str(filesavedir / "Rbin_centers_*.pkl")))
    with open(filesavename, "rb") as f:
        Rbins = pickle.load(f)
    dict['Rbins'] = Rbins

In [23]:
print(dicts[0].keys())

dict_keys(['Force', 'ForceLeft', 'ForceRight', 'PtlMinIdcs', 'count', 'midplane-count', 'midplane-dens', 'SigmaH2Gas', 'SigmaGas', 'Kappa', 'Ptot', 'Ptherm', 'Pturb', 'SigmaSFR', 'Rbins'])


In [22]:
# save each dictionary as a pickle
for tag, dict in zip(tags, dicts):
    filesavedir = Path(config[tag]['ROOT_DIR']) / config[tag]['SUBDIR']
    filesavename = filesavedir / f"props_3D_{tag}.pkl"
    with open(filesavename, "wb") as f:
        pickle.dump(dict, f)
    print(f"Saved: {filesavename}")

Saved: /n/holystore01/LABS/itc_lab/Users/sjeffreson/ETGs/vlM-output/props_3D_ETG-vlM.pkl
Saved: /n/holystore01/LABS/itc_lab/Users/sjeffreson/ETGs/lowM-output/props_3D_ETG-lowM.pkl
Saved: /n/holystore01/LABS/itc_lab/Users/sjeffreson/ETGs/medM-output/props_3D_ETG-medM.pkl
Saved: /n/holystore01/LABS/itc_lab/Users/sjeffreson/ETGs/hiM-output/props_3D_ETG-hiM.pkl
Saved: /n/holystore01/LABS/loeb_lab/Users/sjeffreson/MW-tracers/props_3D_MW.pkl
Saved: /n/holystore01/LABS/itc_lab/Users/sjeffreson/NGC300/props_3D_NGC300.pkl
